# Project 4: Team 7
## Predicting Congressional Bill Passage
### Extract,Transform, and Load: Data for the current Congress- 118th Congress
- Data is all bills before the House and Senate that have not been voted on. 

#### Import dependecies and read in data:

In [293]:
# Import Dependencies:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from joblib import dump, load
from keras.models import load_model
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [294]:
# Read in currentbills.csv, this is current bills before Congress, none that have been voted on or passed:
# Import from google drive folder:
# Mount google drive to get data:
from google.colab import drive
drive.mount('/content/gdrive')
current_filepath = "/content/gdrive/MyDrive/DataClassNotebooks/Project-4/Resources/currentbills.csv"
current_df = pd.read_csv(current_filepath)
# current_df = pd.read_csv('../Resources/currentbills.csv')
# current_df = pd.read_csv('https://raw.githubusercontent.com/JJERANEK/Project-4/main/Resources/currentbills.csv')

# Split data into raw df for the House and Senata data
df_house = current_df[current_df['Legislation Number'].str.contains("H.J|H.R.")==True]
df_house.reset_index(drop=True)
df_senate = current_df[current_df['Legislation Number'].str.contains("S.J|S.")==True]
df_senate.reset_index(drop=True)

# Select only the columns that will be needed:
df_house = df_house[['Legislation Number', 'Congress', 'Title', 'Sponsor',
       'Date of Introduction', 'Number of Cosponsors', 'Committees',
       'Latest Action', 'Latest Action Date', 'Subject']]
df_senate = df_senate[['Legislation Number', 'Congress', 'Title', 'Sponsor',
       'Date of Introduction', 'Number of Cosponsors', 'Committees',
       'Latest Action', 'Latest Action Date', 'Subject']]

# Check master df:
current_df.head()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


<ipython-input-294-364340800247>:7: DtypeWarning: Columns (76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,295,296,297,298,299,300) have mixed types. Specify dtype option on import or set low_memory=False.
  current_df = pd.read_csv(current_filepath)


,Legislation Number,URL,Congress,Title,Amends Bill,Sponsor,Date Offered,Date of Introduction,Number of Cosponsors,Date Submitted,...,Subject.43,Subject.44,Subject.45,Subject.46,Subject.47,Subject.48,Subject.49,Subject.50,Subject.51,Subject.52
0,H.R. 1667,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),To require the Secretary of Agriculture to ide...,NaN,"Westerman, Bruce [Rep.-R-AR-4]",NaN,03/17/2023,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,H.R. 1666,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),To amend title XVIII to protect patient access...,NaN,"Wenstrup, Brad R. [Rep.-R-OH-2]",NaN,03/17/2023,3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,H.R. 1665,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),To direct the Secretary of Transportation to e...,NaN,"Velazquez, Nydia M. [Rep.-D-NY-7]",NaN,03/17/2023,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,H.R. 1664,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),To require the Board of Governors of the Feder...,NaN,"Torres, Ritchie [Rep.-D-NY-15]",NaN,03/17/2023,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,H.R. 1663,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),To require the Secretary of the Treasury to de...,NaN,"Torres, Ritchie [Rep.-D-NY-15]",NaN,03/17/2023,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [295]:
#Get list of cosponsor columns
cosponsor_cols = [col for col in current_df.columns if 'Cosponsor' in col]
cosponsor_cols.remove('Number of Cosponsors')
print(cosponsor_cols)

['Cosponsor', 'Cosponsor.1', 'Cosponsor.2', 'Cosponsor.3', 'Cosponsor.4', 'Cosponsor.5', 'Cosponsor.6', 'Cosponsor.7', 'Cosponsor.8', 'Cosponsor.9', 'Cosponsor.10', 'Cosponsor.11', 'Cosponsor.12', 'Cosponsor.13', 'Cosponsor.14', 'Cosponsor.15', 'Cosponsor.16', 'Cosponsor.17', 'Cosponsor.18', 'Cosponsor.19', 'Cosponsor.20', 'Cosponsor.21', 'Cosponsor.22', 'Cosponsor.23', 'Cosponsor.24', 'Cosponsor.25', 'Cosponsor.26', 'Cosponsor.27', 'Cosponsor.28', 'Cosponsor.29', 'Cosponsor.30', 'Cosponsor.31', 'Cosponsor.32', 'Cosponsor.33', 'Cosponsor.34', 'Cosponsor.35', 'Cosponsor.36', 'Cosponsor.37', 'Cosponsor.38', 'Cosponsor.39', 'Cosponsor.40', 'Cosponsor.41', 'Cosponsor.42', 'Cosponsor.43', 'Cosponsor.44', 'Cosponsor.45', 'Cosponsor.46', 'Cosponsor.47', 'Cosponsor.48', 'Cosponsor.49', 'Cosponsor.50', 'Cosponsor.51', 'Cosponsor.52', 'Cosponsor.53', 'Cosponsor.54', 'Cosponsor.55', 'Cosponsor.56', 'Cosponsor.57', 'Cosponsor.58', 'Cosponsor.59', 'Cosponsor.60', 'Cosponsor.61', 'Cosponsor.62', 'Co

In [296]:
# Create new df with cosponsor columns
cosponsors_df = current_df[cosponsor_cols]
cosponsors_df.head()

,Cosponsor,Cosponsor.1,Cosponsor.2,Cosponsor.3,Cosponsor.4,Cosponsor.5,Cosponsor.6,Cosponsor.7,Cosponsor.8,Cosponsor.9,...,Cosponsor.224,Cosponsor.225,Cosponsor.226,Cosponsor.227,Cosponsor.228,Cosponsor.229,Cosponsor.230,Cosponsor.231,Cosponsor.232,Cosponsor.233
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Carter, Earl L. ""Buddy"" [Rep.-R-GA-1]","Sewell, Terri A. [Rep.-D-AL-7]","Tonko, Paul [Rep.-D-NY-20]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Espaillat, Adriano [Rep.-D-NY-13]","Meng, Grace [Rep.-D-NY-6]","Goldman, Daniel S. [Rep.-D-NY-10]","Clarke, Yvette D. [Rep.-D-NY-9]",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [297]:
# Add bill and congress for identification, number of cosponsors to ensure party counts total correct
cosponsors_df.insert(0, "Legislation Number", current_df['Legislation Number'])
cosponsors_df.insert(1, "Congress", current_df['Congress'])
cosponsors_df.insert(2, "Number of Cosponsors", current_df['Number of Cosponsors'])
cosponsors_df.head()

,Legislation Number,Congress,Number of Cosponsors,Cosponsor,Cosponsor.1,Cosponsor.2,Cosponsor.3,Cosponsor.4,Cosponsor.5,Cosponsor.6,...,Cosponsor.224,Cosponsor.225,Cosponsor.226,Cosponsor.227,Cosponsor.228,Cosponsor.229,Cosponsor.230,Cosponsor.231,Cosponsor.232,Cosponsor.233
0,H.R. 1667,118th Congress (2023-2024),0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,H.R. 1666,118th Congress (2023-2024),3,"Carter, Earl L. ""Buddy"" [Rep.-R-GA-1]","Sewell, Terri A. [Rep.-D-AL-7]","Tonko, Paul [Rep.-D-NY-20]",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,H.R. 1665,118th Congress (2023-2024),4,"Espaillat, Adriano [Rep.-D-NY-13]","Meng, Grace [Rep.-D-NY-6]","Goldman, Daniel S. [Rep.-D-NY-10]","Clarke, Yvette D. [Rep.-D-NY-9]",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,H.R. 1664,118th Congress (2023-2024),0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,H.R. 1663,118th Congress (2023-2024),0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [298]:
# Count cosponsor dems per row and add to df
cosponsor_dems = cosponsors_df.astype(str).apply(lambda x: x.str.contains('-D-')).sum(axis=1)
cosponsors_df['Cosponsor Dems'] = cosponsor_dems


<ipython-input-298-bf072d028ff3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cosponsors_df['Cosponsor Dems'] = cosponsor_dems


In [299]:
# Count cosponsor Reps per row and add to df
cosponsor_reps = cosponsors_df.astype(str).apply(lambda x: x.str.contains('-R-')).sum(axis=1)
cosponsors_df['Cosponsor Reps'] = cosponsor_reps

<ipython-input-299-d90870787459>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cosponsors_df['Cosponsor Reps'] = cosponsor_reps


In [300]:
# Count cosponsor Independent per row and add to df
cosponsor_ind = cosponsors_df.astype(str).apply(lambda x: x.str.contains('-I-')).sum(axis=1)
cosponsors_df['Cosponsor Ind'] = cosponsor_ind

<ipython-input-300-bd71d2f0a538>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cosponsors_df['Cosponsor Ind'] = cosponsor_ind


In [301]:
# get state for each cosponsor
for col in cosponsor_cols:
    cosponsors_df[col].update(cosponsors_df[col].str.split('-').str[2])
cosponsors_df.head()

<ipython-input-301-7bd367cbb2f6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cosponsors_df[col].update(cosponsors_df[col].str.split('-').str[2])


,Legislation Number,Congress,Number of Cosponsors,Cosponsor,Cosponsor.1,Cosponsor.2,Cosponsor.3,Cosponsor.4,Cosponsor.5,Cosponsor.6,...,Cosponsor.227,Cosponsor.228,Cosponsor.229,Cosponsor.230,Cosponsor.231,Cosponsor.232,Cosponsor.233,Cosponsor Dems,Cosponsor Reps,Cosponsor Ind
0,H.R. 1667,118th Congress (2023-2024),0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
1,H.R. 1666,118th Congress (2023-2024),3,GA,AL,NY,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,0
2,H.R. 1665,118th Congress (2023-2024),4,NY,NY,NY,NY,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,0,0
3,H.R. 1664,118th Congress (2023-2024),0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
4,H.R. 1663,118th Congress (2023-2024),0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0


In [302]:
# remove any remaining brackets
cosponsors_df[cosponsor_cols] = cosponsors_df[cosponsor_cols].replace({']':''}, regex=True)


<ipython-input-302-9fcbba2ad5be>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cosponsors_df[cosponsor_cols] = cosponsors_df[cosponsor_cols].replace({']':''}, regex=True)


In [303]:
#get count of unique states
cosponsor_states = cosponsors_df[cosponsor_cols].nunique(axis=1)
cosponsors_df['Cosponsor States'] = cosponsor_states


<ipython-input-303-67ffd9929953>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cosponsors_df['Cosponsor States'] = cosponsor_states


In [304]:
# Inspect:
cosponsors_df.head()

,Legislation Number,Congress,Number of Cosponsors,Cosponsor,Cosponsor.1,Cosponsor.2,Cosponsor.3,Cosponsor.4,Cosponsor.5,Cosponsor.6,...,Cosponsor.228,Cosponsor.229,Cosponsor.230,Cosponsor.231,Cosponsor.232,Cosponsor.233,Cosponsor Dems,Cosponsor Reps,Cosponsor Ind,Cosponsor States
0,H.R. 1667,118th Congress (2023-2024),0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
1,H.R. 1666,118th Congress (2023-2024),3,GA,AL,NY,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2,1,0,3
2,H.R. 1665,118th Congress (2023-2024),4,NY,NY,NY,NY,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4,0,0,1
3,H.R. 1664,118th Congress (2023-2024),0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
4,H.R. 1663,118th Congress (2023-2024),0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0


In [305]:
# Create clean df with cosponsor counts
clean_cosponsor_df = cosponsors_df[['Legislation Number','Congress','Number of Cosponsors','Cosponsor Dems','Cosponsor Reps','Cosponsor Ind', 'Cosponsor States']].reset_index(drop=True)


In [306]:
# Join clean cosponsor df with house df
house_df = pd.merge(clean_cosponsor_df, df_house, how='inner', on=['Legislation Number', 'Congress'])
house_df = house_df.drop(columns='Number of Cosponsors_y').rename(columns={'Number of Cosponsors_x': 'Number of Cosponsors'})


In [307]:
# Join clean cosponsor df with senate df
senate_df = pd.merge(clean_cosponsor_df, df_senate, how='inner', on=['Legislation Number', 'Congress'])
senate_df = senate_df.drop(columns='Number of Cosponsors_y').rename(columns={'Number of Cosponsors_x': 'Number of Cosponsors'})


In [308]:
# Concat house and senate dfs to finish cleaning
frames = [house_df, senate_df]
congress_df = pd.concat(frames).reset_index(drop=True)
congress_df.head()

,Legislation Number,Congress,Number of Cosponsors,Cosponsor Dems,Cosponsor Reps,Cosponsor Ind,Cosponsor States,Title,Sponsor,Date of Introduction,Committees,Latest Action,Latest Action Date,Subject
0,H.R. 1667,118th Congress (2023-2024),0,0,0,0,0,To require the Secretary of Agriculture to ide...,"Westerman, Bruce [Rep.-R-AR-4]",03/17/2023,House - Natural Resources,Referred to the House Committee on Natural Res...,03/17/2023,NaN
1,H.R. 1666,118th Congress (2023-2024),3,2,1,0,3,To amend title XVIII to protect patient access...,"Wenstrup, Brad R. [Rep.-R-OH-2]",03/17/2023,"House - Energy and Commerce, Ways and Means",Referred to the Committee on Energy and Commer...,03/17/2023,NaN
2,H.R. 1665,118th Congress (2023-2024),4,4,0,0,1,To direct the Secretary of Transportation to e...,"Velazquez, Nydia M. [Rep.-D-NY-7]",03/17/2023,House - Transportation and Infrastructure,Referred to the House Committee on Transportat...,03/17/2023,NaN
3,H.R. 1664,118th Congress (2023-2024),0,0,0,0,0,To require the Board of Governors of the Feder...,"Torres, Ritchie [Rep.-D-NY-15]",03/17/2023,House - Financial Services,Referred to the House Committee on Financial S...,03/17/2023,NaN
4,H.R. 1663,118th Congress (2023-2024),0,0,0,0,0,To require the Secretary of the Treasury to de...,"Torres, Ritchie [Rep.-D-NY-15]",03/17/2023,House - Financial Services,Referred to the House Committee on Financial S...,03/17/2023,NaN


In [309]:
# Strip numbers and change Legislation Number to Bill Type
congress_df['Legislation Number'] = congress_df['Legislation Number'].str.replace('\d+', '')
congress_df = congress_df.rename(columns = {"Legislation Number": "Bill Type"})
# Get number of congress only, column 2
congress_df['Congress'] = congress_df['Congress'].str[:3]
# Cast as int64:
congress_df['Congress'] = congress_df['Congress'].astype(int)

<ipython-input-309-3e35feaf2316>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  congress_df['Legislation Number'] = congress_df['Legislation Number'].str.replace('\d+', '')


In [310]:
# extract party and state into new column for sponsor
new = congress_df["Sponsor"].str.split("[", n = 1, expand = True)
congress_df['Sponsor Split']= new[1]
congress_df.drop(columns =["Sponsor"], inplace = True)


In [311]:
# Sponsor title, sponsor state, sponsor party in new columns
new2 = congress_df["Sponsor Split"].str.split("-", n = 3, expand = True)
congress_df['Sponsor Title']= new2[0]
congress_df['Sponsor Party']= new2[1]
congress_df['Sponsor State']= new2[2]
congress_df = congress_df.drop(columns={'Sponsor Split'})


In [312]:
# Create the month of bill introduction:
congress_df['Date of Introduction'] = pd.to_datetime(congress_df['Date of Introduction'])
congress_df['Month Introduced'] = pd.DatetimeIndex(congress_df['Date of Introduction']).month
congress_df = congress_df.drop(columns={'Date of Introduction'})
# Drop unneeded columns:
congress_df = congress_df.drop(columns={'Latest Action Date'})
# Take out extra brackets in statecolumn:
congress_df['Sponsor State'] = congress_df['Sponsor State'].replace({']':''}, regex=True)

## Save whole congress dataset up to this point:

In [313]:
# Save whole cleaned dataset:
# congress_df.to_csv('../output/congress_predict_clean.csv')

### Split the data by House and Senate:

In [314]:
# Split into house and senate dfs:
house_cleaned = congress_df[congress_df['Bill Type'].str.contains("H.J|H")==True]
senate_cleaned = congress_df[congress_df['Bill Type'].str.contains("S.J|S.")==True]
senate_cleaned = senate_cleaned.reset_index(drop=True)
house_cleaned.head()

,Bill Type,Congress,Number of Cosponsors,Cosponsor Dems,Cosponsor Reps,Cosponsor Ind,Cosponsor States,Title,Committees,Latest Action,Subject,Sponsor Title,Sponsor Party,Sponsor State,Month Introduced
0,H.R.,118,0,0,0,0,0,To require the Secretary of Agriculture to ide...,House - Natural Resources,Referred to the House Committee on Natural Res...,NaN,Rep.,R,AR,3
1,H.R.,118,3,2,1,0,3,To amend title XVIII to protect patient access...,"House - Energy and Commerce, Ways and Means",Referred to the Committee on Energy and Commer...,NaN,Rep.,R,OH,3
2,H.R.,118,4,4,0,0,1,To direct the Secretary of Transportation to e...,House - Transportation and Infrastructure,Referred to the House Committee on Transportat...,NaN,Rep.,D,NY,3
3,H.R.,118,0,0,0,0,0,To require the Board of Governors of the Feder...,House - Financial Services,Referred to the House Committee on Financial S...,NaN,Rep.,D,NY,3
4,H.R.,118,0,0,0,0,0,To require the Secretary of the Treasury to de...,House - Financial Services,Referred to the House Committee on Financial S...,NaN,Rep.,D,NY,3


### House data cleaning:

In [315]:
# Committee column recoding to indicator variables:
# Create a list of committees for the House:
house_committees_lst = ["Agriculture", "Appropriations", "Armed Services", "Budget", "Education and the Workforce", "Energy and Commerce", "Ethics", "Financial Services", 
                        "Foreign Affairs", "Homeland Security", "House Administration", "Judiciary", "Natural Resources", 
                        "Oversight and Accountability", "Rules", "Science, Space, and Technology", "Small Business", "Transportation and Infrastructure", 
                        "Veterans' Affairs", "Ways and Means", "Intelligence", "Printing", "Taxation", "Library", "Economic"]
# Run a for loop to set each committee name to a new column and make a dummy var (case=False makes the str.contains case insensitive)):
for comm in house_committees_lst:
    house_cleaned[comm] = np.where(house_cleaned['Committees'].str.contains(comm, case=False), 1, 0)

<ipython-input-315-65e5480ed904>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  house_cleaned[comm] = np.where(house_cleaned['Committees'].str.contains(comm, case=False), 1, 0)
<ipython-input-315-65e5480ed904>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  house_cleaned[comm] = np.where(house_cleaned['Committees'].str.contains(comm, case=False), 1, 0)
<ipython-input-315-65e5480ed904>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [316]:
# Latest Action coding:
# Get "referred to the committee" and name it "bill_referred_committee" set as dummy var:
house_cleaned["bill_referred_committee"] = np.where(house_cleaned['Latest Action'].str.contains("referred to the committee", case=False), 1, 0)
print(house_cleaned["bill_referred_committee"].value_counts())
print(f'So far the 118th House has referred {(round((331/1365)*100,2))}% of laws introduced to a committee.')

0    1365
1     331
Name: bill_referred_committee, dtype: int64
So far the 118th House has referred 24.25% of laws introduced to a committee.


<ipython-input-316-6ff093dc9306>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  house_cleaned["bill_referred_committee"] = np.where(house_cleaned['Latest Action'].str.contains("referred to the committee", case=False), 1, 0)


In [317]:
# Drop the non-beneficial columns: 
house_cleaned = house_cleaned.drop(["Bill Type", "Title", "Committees", "Latest Action" ], axis='columns')
house_cleaned.head()

,Congress,Number of Cosponsors,Cosponsor Dems,Cosponsor Reps,Cosponsor Ind,Cosponsor States,Subject,Sponsor Title,Sponsor Party,Sponsor State,...,Small Business,Transportation and Infrastructure,Veterans' Affairs,Ways and Means,Intelligence,Printing,Taxation,Library,Economic,bill_referred_committee
0,118,0,0,0,0,0,NaN,Rep.,R,AR,...,0,0,0,0,0,0,0,0,0,0
1,118,3,2,1,0,3,NaN,Rep.,R,OH,...,0,0,0,1,0,0,0,0,0,1
2,118,4,4,0,0,1,NaN,Rep.,D,NY,...,0,1,0,0,0,0,0,0,0,0
3,118,0,0,0,0,0,NaN,Rep.,D,NY,...,0,0,0,0,0,0,0,0,0,0
4,118,0,0,0,0,0,NaN,Rep.,D,NY,...,0,0,0,0,0,0,0,0,0,0


In [318]:
# Look at Subject value counts for binning
Subject_counts = house_cleaned["Subject"].value_counts()
print(f'Count of values for Subject column: \n{Subject_counts}')

Count of values for Subject column: 
Health                                                101
Government Operations and Politics                     95
Armed Forces and National Security                     93
Crime and Law Enforcement                              81
Taxation                                               79
                                                     ... 
Criminal investigation, prosecution, interrogation      1
Firearms and explosives                                 1
Civil actions and liability                             1
Health facilities and institutions                      1
Age discrimination                                      1
Name: Subject, Length: 118, dtype: int64


In [319]:
# Now change cutoff value to get  bins for Subject
# Cutoff value of 2 to bin: 
# use the variable name `Subject_types_to_replace`
Subject_types_to_replace = list(Subject_counts[Subject_counts<3].index)

# Replace in dataframe
for sub in Subject_types_to_replace:
    house_cleaned['Subject'] = house_cleaned['Subject'].replace(sub,"Other")

# Check to make sure binning was successful
print(len(house_cleaned['Subject'].value_counts()))
house_df['Subject'].value_counts()
# 31 Bins

35


Health                                                101
Government Operations and Politics                     95
Armed Forces and National Security                     93
Crime and Law Enforcement                              81
Taxation                                               79
                                                     ... 
Criminal investigation, prosecution, interrogation      1
Firearms and explosives                                 1
Civil actions and liability                             1
Health facilities and institutions                      1
Age discrimination                                      1
Name: Subject, Length: 118, dtype: int64

In [320]:
# Convert categorical data to numeric with `pd.get_dummies`
house_cleaned = pd.get_dummies(house_cleaned,dtype=float)
house_cleaned.head()

,Congress,Number of Cosponsors,Cosponsor Dems,Cosponsor Reps,Cosponsor Ind,Cosponsor States,Month Introduced,Agriculture,Appropriations,Armed Services,...,Sponsor State_TN,Sponsor State_TX,Sponsor State_UT,Sponsor State_VA,Sponsor State_VI,Sponsor State_VT,Sponsor State_WA,Sponsor State_WI,Sponsor State_WV,Sponsor State_WY
0,118,0,0,0,0,0,3,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,118,3,2,1,0,3,3,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,118,4,4,0,0,1,3,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,118,0,0,0,0,0,3,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,118,0,0,0,0,0,3,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [321]:
# Load in house_scaler from house model output: house_scaler.bin
from google.colab import files

output_filepath = "/content/gdrive/MyDrive/DataClassNotebooks/Project-4/output"
house_scaler=load(f'{output_filepath}/house_scaler.bin')
# house_scaler=load('../output/house_scaler.bin')
# Scale the house_cleaned data with saved house_scaler:
house_scaled = house_scaler.transform(house_cleaned)


/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [322]:
# Load in saved house_model.h5:
house_model = load_model(f'{output_filepath}/house_model.h5')

In [323]:
# Make predictions with saved house_model on scale house data- house_scaled:
house_cleaned["house_probabilities"] = house_model.predict(house_scaled)
house_cleaned["house_predictions"] = np.argmax(house_model.predict(house_scaled),axis=1)
house_cleaned["house_predictions"].unique()

53/53 [==============================] - 0s 3ms/step


array([0])

## Save clean house dataset with prediction and probability columns: 

In [324]:
# Save house cleaned dataset using current Congress and added predictions:
# house_cleaned.to_csv(f'{output_filepath}/house_predict_current.csv', index=False)

### Senate Cleaning:

In [325]:
# Committee column recoding to indicator variables:
# Create a list of committees for the Senate:
senate_committees_lst = ["Agriculture, Nutrition, and Forestry", "Appropriations", "Armed Services", "Banking, Housing, and Urban Affairs", "Budget", 
                         "Commerce, Science, and Transportation", "Energy and Natural Resources", "Environment and Public Works", "Finance", 
                         "Foreign Relations", "Health, Education, Labor, and Pensions", "Homeland Security and Governmental Affairs","Judiciary", 
                         "Rules and Administration", "Small Business and Entrepreneurship", "Veterans Affairs", "International Narcotics Control", 
                         "Ethics", "Indian Affairs", "Intelligence", "Printing", "Taxation", "Library", "Economic"]
# Run a for loop to set each committee name to a new column and make a dummy var (case=False makes the str.contains case insensitive)):
for comm in senate_committees_lst:
    senate_cleaned[comm] = np.where(senate_cleaned['Committees'].str.contains(comm, case=False), 1, 0)

senate_cleaned.head()

,Bill Type,Congress,Number of Cosponsors,Cosponsor Dems,Cosponsor Reps,Cosponsor Ind,Cosponsor States,Title,Committees,Latest Action,...,Small Business and Entrepreneurship,Veterans Affairs,International Narcotics Control,Ethics,Indian Affairs,Intelligence,Printing,Taxation,Library,Economic
0,S.,118,1,0,1,0,1,A bill to prohibit the receipt of Federal fund...,"Senate - Commerce, Science, and Transportation",Read twice and referred to the Committee on Co...,...,0,0,0,0,0,0,0,0,0,0
1,S.,118,1,0,1,0,1,A bill to direct the Secretary of Labor to mod...,Senate - Judiciary,Read twice and referred to the Committee on th...,...,0,0,0,0,0,0,0,0,0,0
2,S.,118,1,0,1,0,1,"A bill to improve recreation opportunities on,...",Senate - Energy and Natural Resources,Read twice and referred to the Committee on En...,...,0,0,0,0,0,0,0,0,0,0
3,S.,118,0,0,0,0,0,A bill to identify social media entities under...,"Senate - Banking, Housing, and Urban Affairs",Read twice and referred to the Committee on Ba...,...,0,0,0,0,0,0,0,0,0,0
4,S.,118,5,2,3,0,5,A bill to amend section 7014 of the Elementary...,"Senate - Health, Education, Labor, and Pensions",Read twice and referred to the Committee on He...,...,0,0,0,0,0,0,0,0,0,0


In [326]:
# Latest Action coding:
# Get "referred to the committee" and name it "bill_referred_committee" set as dummy var:
senate_cleaned["bill_referred_committee"] = np.where(senate_cleaned['Latest Action'].str.contains("referred to the committee", case=False), 1, 0)
print(senate_cleaned["bill_referred_committee"].value_counts())
print(f'For the 118th Senate so far {(round((41/849)*100,2))}% of bills introduced in the Senate were referred to a committee.')

1    849
0     41
Name: bill_referred_committee, dtype: int64
For the 118th Senate so far 4.83% of bills introduced in the Senate were referred to a committee.


In [327]:
# Look at Subject value counts for binning
Subject_counts_sen = senate_cleaned["Subject"].value_counts()
print(f'Count of values for Subject column: \n{Subject_counts_sen}')

Count of values for Subject column: 
International Affairs                               47
Health                                              47
Crime and Law Enforcement                           42
Taxation                                            41
Government Operations and Politics                  40
                                                    ..
Neurological disorders                               1
Law enforcement officers                             1
Asia                                                 1
Housing and Community Development                    1
Elections, voting, political campaign regulation     1
Name: Subject, Length: 63, dtype: int64


In [328]:
# Now change cutoff value to get  bins for Subject
# Cutoff value of 100 to bin: 
# use the variable name `Subject_types_to_replace_sen`
Subject_types_to_replace_sen = list(Subject_counts_sen[Subject_counts_sen<5].index)

# # Replace in dataframe
for sub in Subject_types_to_replace_sen:
    senate_cleaned['Subject'] = senate_cleaned['Subject'].replace(sub,"Other")

# Check to make sure binning was successful
print(len(senate_cleaned['Subject'].value_counts()))
senate_cleaned['Subject'].value_counts()
# 27 Bins

27


International Affairs                          47
Health                                         47
Other                                          43
Crime and Law Enforcement                      42
Taxation                                       41
Government Operations and Politics             40
Armed Forces and National Security             36
Education                                      27
Energy                                         24
Public Lands and Natural Resources             23
Immigration                                    22
Agriculture and Food                           21
Labor and Employment                           18
Commerce                                       17
Finance and Financial Sector                   16
Congress                                       13
Foreign Trade and International Finance        11
Transportation and Public Works                11
Economics and Public Finance                   10
Environmental Protection                       10


In [329]:
# Drop the non-beneficial columns: 
senate_cleaned = senate_cleaned.drop(["Bill Type", "Title", "Committees", "Latest Action" ], axis='columns')
senate_cleaned.head()

,Congress,Number of Cosponsors,Cosponsor Dems,Cosponsor Reps,Cosponsor Ind,Cosponsor States,Subject,Sponsor Title,Sponsor Party,Sponsor State,...,Veterans Affairs,International Narcotics Control,Ethics,Indian Affairs,Intelligence,Printing,Taxation,Library,Economic,bill_referred_committee
0,118,1,0,1,0,1,NaN,Sen.,R,FL,...,0,0,0,0,0,0,0,0,0,1
1,118,1,0,1,0,1,NaN,Sen.,D,GA,...,0,0,0,0,0,0,0,0,0,1
2,118,1,0,1,0,1,NaN,Sen.,D,WV,...,0,0,0,0,0,0,0,0,0,1
3,118,0,0,0,0,0,NaN,Sen.,R,AR,...,0,0,0,0,0,0,0,0,0,1
4,118,5,2,3,0,5,NaN,Sen.,D,NM,...,0,0,0,0,0,0,0,0,0,1


In [330]:
# Convert categorical data to numeric with `pd.get_dummies`
senate_cleaned = pd.get_dummies(senate_cleaned,dtype=float)
senate_cleaned.head()
# Creates a df of 113 columns

,Congress,Number of Cosponsors,Cosponsor Dems,Cosponsor Reps,Cosponsor Ind,Cosponsor States,Month Introduced,"Agriculture, Nutrition, and Forestry",Appropriations,Armed Services,...,Sponsor State_SD,Sponsor State_TN,Sponsor State_TX,Sponsor State_UT,Sponsor State_VA,Sponsor State_VT,Sponsor State_WA,Sponsor State_WI,Sponsor State_WV,Sponsor State_WY
0,118,1,0,1,0,1,3,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,118,1,0,1,0,1,3,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,118,1,0,1,0,1,3,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,118,0,0,0,0,0,3,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,118,5,2,3,0,5,3,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [337]:
# Drop last column "Sponsor State_WY":
senate_cleaned = senate_cleaned.drop("Sponsor State_WY", axis=1)


In [333]:
# Load in house_scaler from house model output: house_scaler.bin
from google.colab import files

output_filepath = "/content/gdrive/MyDrive/DataClassNotebooks/Project-4/output"
senate_scaler=load(f'{output_filepath}/senate_scaler.bin')
# senate_scaler=load('../output/senate_scaler.bin')
# Scale the house_cleaned data with saved house_scaler:
senate_scaled = senate_scaler.transform(senate_cleaned)

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [334]:
# Load in saved senate_model.h5:
senate_model = load_model(f'{output_filepath}/senate_model.h5')

In [339]:
# Make predictions with saved senate_model on scale senate data- senate_scaled:
senate_cleaned["senate_probabilities"] = senate_model.predict(senate_scaled)
senate_cleaned["senate_predictions"] = np.argmax(senate_model.predict(senate_scaled),axis=1)
# senate_cleaned["senate_probabilities"].unique()

28/28 [==============================] - 0s 2ms/step


## Save clean senate dataset: 

In [340]:
# Save senate cleaned dataset using current Congress and added predictions:
# senate_cleaned.to_csv(f'{output_filepath}/senate_predict_current.csv', index=False)